# Conducción de Calor en 2D

**Objetivo General**
- Resolver numérica y computacionalmente la ecuación de conducción de calor en dos dimensiones.

**Objetivos particulares**
- Definir los parámetros físicos y numéricos.
- Definir la malla del dominio.
- Definir la temperatura inicial junto con sus condiciones de frontera y graficarla sobre la malla.
- Definir el sistema lineal y resolverlo.
- Graficar la solución.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import hecompa_FD2D as fd2
import macti.visual as mvis

# TODO: hacer este cambio en macti.visual
plt.rcParams['axes.grid'] = False

## Parámetros físicos y numéricos

In [ ]:
# Tamaño del dominio
Lx = 1.0
Ly = 1.0

# Número de nodos en cada eje
Nx = 4
Ny = 4

# Número total de nodos en cada eje incluyendo las fronteras
NxT = Nx + 2
NyT = Ny + 2

# Número total de nodos
NT = NxT * NyT

# Número total de incógnitas
N = Nx * Ny

# Tamaño de la malla en cada dirección
hx = Lx / (Nx+1)
hy = Ly / (Ny+1)

# Coordenadas de la malla
xn = np.linspace(0,Lx,NxT)
yn = np.linspace(0,Ly,NyT)

# Generación de una rejilla
xg, yg = np.meshgrid(xn, yn, indexing='ij')

In [ ]:
print('Total de nodos en x = {}, en y = {}'.format(NxT, NyT))
print('Total de incógnitas = {}'.format(N))
print('Coordenadas en x : {}'.format(xn))
print('Coordenadas en y : {}'.format(yn))
print('hx = {}, hy = {}'.format(hx, hy))

### Graficación de la malla del dominio

In [ ]:
fig = plt.figure()
ax = plt.gca()

# Ejecutamos la función plot_mesh(...)
fd2.plot_mesh(ax, xg, yg)

# Dibujamos el recuadro con la función plot_fame(...)
fd2.plot_frame(ax, xn, yn)

## Campo de temperaturas y sus condiciones de frontera

In [ ]:
# Definición de un campo escalar en cada punto de la malla
T = np.zeros((NxT, NyT))

# Condiciones de frontera
TB = 1.0
TT = -1.0

T[0 , :] = 0.0 # LEFT
T[-1, :] = 0.0 # RIGHT
T[: , 0] = TB  # BOTTOM
T[: ,-1] = TT  # TOP

print('Campo escalar T ({}):\n {}'.format(T.shape, T))

### Graficación del campo escalar sobre la malla

In [ ]:
fig = plt.figure()
ax = plt.gca()
cax = fd2.set_canvas(ax, Lx, Ly)

c = ax.contourf(xg, yg, T, levels=50, cmap='inferno')
fd2.plot_mesh(ax, xg, yg)

fig.colorbar(c, cax=cax, ticks=[-0.9, 0.0, 0.9])
plt.show()

## Sistema lineal

In [ ]:
# La matriz del sistema. Usamos la función predefinida buildMatrix2D()
A = fd2.buildMatrix2D(Nx,Ny,-4)
A

In [ ]:
# RHS
b = np.zeros((Nx,Ny))
b[:, 0] -= TB  # BOTTOM
b[:,-1] -= TT  # TOP
b

## Solución del sistema

Revisamos el formato del vector b

In [ ]:
b.shape

El vector debe ser de una sola dimensión:

In [ ]:
b.flatten()

In [ ]:
# Calculamos la solución.
T_temp = np.linalg.solve(A, b.flatten())
T_temp

In [ ]:
T_temp.shape

Colocamos la solución en el campo escalar T de manera adecuada

In [ ]:
T[1:-1,1:-1] = T_temp.reshape(Nx,Ny)
T

### Gráfica de la solución

In [ ]:
fig = plt.figure()
ax = plt.gca()
cax = fd2.set_canvas(ax, Lx, Ly)

c = ax.contourf(xg, yg, T, levels=50, cmap='inferno')
fd2.plot_mesh(ax, xg, yg)

fig.colorbar(c, cax=cax, ticks=[-0.9, 0.0, 0.9])
plt.show()